In [2]:
regions_dict = {
  1: 'Ленинградская область',
  3138: 'Санкт-Петербург',
  3268: 'Москва',
  3427: 'Волгоградская область',
  3902: 'Тульская область',
  4013: 'Омская область',
  4437: 'Краснодарский край',
  4882: 'Тюменская область',
  5188: 'Московская область',
}

In [3]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
from townsnet import Region

for region_id, region_name in regions_dict.items():
  region_path = f"data/regions/{region_id}/"

  districts = gpd.read_file(region_path + 'districts.geojson')
  settlements = gpd.read_file(region_path + 'settlements.geojson')
  towns = gpd.read_file(region_path + 'towns.geojson')
  adj_mx = pd.read_pickle(region_path + 'adj_mx.pickle')
  
  crs = towns.estimate_utm_crs()
  region = Region(
    districts.to_crs(crs), 
    settlements.to_crs(crs), 
    towns.to_crs(crs), 
    adj_mx
  )
  for service_type in tqdm(region.service_types):
    try:
      gdf = gpd.read_file(region_path + f'services/{service_type.name}.geojson').to_crs(crs)
      gdf = region.match_services_towns(gdf)
      region.update_services(service_type, gdf)
    except:
      ...
  
  region.to_pickle(region_path + f'{region_id}.pickle')
  

100%|██████████| 45/45 [04:36<00:00,  6.15s/it]


DriverError: data/regions/5188/settlements.geojson: No such file or directory

## Выгрузка иерархии показателей

In [16]:
infrastructure_dict = {
  'EDUCATION': 'Обеспеченность объектами образования',
  'HEALTHCARE': 'Обеспеченность объектами здравоохранения',
  'COMMERCE': 'Обеспеченность объектами торговли',
  'CATERING': 'Обеспеченность объектами общепита',
  'LEISURE': 'Обеспеченность объектами досуга',
  'RECREATION': 'Обеспеченность объектами рекреации',
  'SPORT': "Обеспеченность объектами спорта",
  'SERVICE': "Обеспеченность объектами услуг",
  'TRANSPORT': "Обеспеченность объектами транспорта",
  'SAFENESS': "Обеспеченность объектами безопасности",
}

st_df = region.get_service_types_df()
for i,infrastructure in enumerate(st_df.infrastructure.unique()):
  print(f"5.{i+1} {infrastructure_dict[infrastructure]}")
  st_sub_df = st_df[st_df.infrastructure == infrastructure]
  for j, st_name in enumerate(st_sub_df.name_ru):
    print(f"5.{i+1}.{j+1} Обеспеченность {st_name}")

5.1 Обеспеченность объектами образования
5.1.1 Обеспеченность детский сад
5.1.2 Обеспеченность школа
5.1.3 Обеспеченность центр детского творчества
5.1.4 Обеспеченность школа искусств
5.2 Обеспеченность объектами здравоохранения
5.2.1 Обеспеченность ФАП / амбулатория
5.2.2 Обеспеченность аптека
5.2.3 Обеспеченность поликлиника / центр семейной медицины
5.2.4 Обеспеченность участковая больница
5.2.5 Обеспеченность городская больница (д/в)
5.2.6 Обеспеченность коммерческая клиника
5.3 Обеспеченность объектами торговли
5.3.1 Обеспеченность продуктовый магазин
5.3.2 Обеспеченность хозяйственный магазин
5.3.3 Обеспеченность ТЦ / супермаркет
5.3.4 Обеспеченность зоомагазин
5.3.5 Обеспеченность ТРК / гипермаркет
5.3.6 Обеспеченность профильный магазин
5.4 Обеспеченность объектами общепита
5.4.1 Обеспеченность кафе / кофейня
5.4.2 Обеспеченность бар / ресторан
5.4.3 Обеспеченность фудкорт
5.5 Обеспеченность объектами досуга
5.5.1 Обеспеченность универсальный зал
5.5.2 Обеспеченность комьюнити-